In [26]:
from kedro_datasets.pandas import ParquetDataset
import pandas as pd

#kobe_raw_dev = catalog.load(name='kobe_raw_dev@parquet')
pd.options.display.max_columns = None
pd.options.display.max_rows = None
dataset = ParquetDataset(filepath='../data/01_raw/dataset_kobe_dev.parquet').load()

In [27]:
dataset = dataset.drop(columns=['action_type','combined_shot_type','game_event_id','game_id','loc_x','loc_y','season','seconds_remaining','shot_type','shot_zone_area','shot_zone_basic','shot_zone_range','team_id','team_name','game_date','matchup','opponent','shot_id'])

In [28]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24271 entries, 0 to 30696
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   lat                24271 non-null  float64
 1   lon                24271 non-null  float64
 2   minutes_remaining  24271 non-null  int64  
 3   period             24271 non-null  int64  
 4   playoffs           24271 non-null  int64  
 5   shot_distance      24271 non-null  int64  
 6   shot_made_flag     20285 non-null  float64
dtypes: float64(3), int64(4)
memory usage: 1.5 MB


In [4]:
#dataset.dropna()

In [5]:
#dataset.info()